# Agenda

1. Pandas, NumPy, and PyArrow
2. Arrow -- what it is
3. PyArrow for file loading/saving
4. PyArrow as a back-end data type
5. Transitioning to PyArrow

# Pandas, NumPy, and PyArrow

NumPy has been around for about 20 years. It gives us the speed and efficiency of C data types, but a Python API. That means we can use Python for numeric computing.

NumPy defines the "NumPy array," which contains a number of values all of the same type. These are all stored in C, which means that (in contrast with normal Python programming), we have to specify not only the type of values, but their lengths, as well.

In [1]:
import numpy as np

In [2]:
a = np.array([10, 20, 30, 40, 50])
a

array([10, 20, 30, 40, 50])

In [3]:
type(a)

numpy.ndarray

In [4]:
a.dtype

dtype('int64')

In [5]:
64 * 5

320

In [6]:
mylist = [10, 20, 30, 40, 50]
import sys
sys.getsizeof(mylist)

104

In [7]:
a = np.array([10, 20, 30, 40, 50], dtype='int8')
a

array([10, 20, 30, 40, 50], dtype=int8)

NumPy is great! But it's also a bit low level. About 15 years ago, Wes McKinney invented Pandas, which is a wrapper around NumPy, giving us a lot of convenience methods. NumPy handles n-dimensional arrays. Pandas has only two main data structures, one is the Series (1D) and also the DataFrame (2D).

Pandas gives us lots of methods for:

- Loading data from different formats
- Storing data in different formats
- Analyzing in many ways
- Cleaning our data
- Visualizing our data